In [ ]:
# Downloading packages
%pip install pandas pymongo cloudscraper python-dotenv tqdm


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# importing libraries
import os
import json
import time
import math
import pandas as pd
from pymongo import MongoClient
from dotenv import load_dotenv
import cloudscraper
from tqdm import tqdm

# Load variables 
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")

YEARS = list(range(2000, 2025))
BASE_URL = "https://africa-energy-portal.org"
FETCH_ENDPOINT = f"{BASE_URL}/get-database-data"

COUNTRIES = [
    "Algeria","Angola","Benin","Botswana","Burkina Faso","Burundi","Cameroon","Cape Verde",
    "Central African Republic","Chad","Comoros","Congo Democratic Republic","Congo Republic",
    "Cote d'Ivoire","Djibouti","Egypt","Equatorial Guinea","Eritrea","Eswatini","Ethiopia",
    "Gabon","Gambia","Ghana","Guinea","Guinea Bissau","Kenya","Lesotho","Liberia","Libya",
    "Madagascar","Malawi","Mali","Mauritania","Mauritius","Morocco","Mozambique","Namibia",
    "Niger","Nigeria","Rwanda","Sao Tome and Principe","Senegal","Seychelles","Sierra Leone",
    "Somalia","South Africa","South Sudan","Sudan","Tanzania","Togo","Tunisia","Uganda",
    "Zambia","Zimbabwe"
]


scraper = cloudscraper.create_scraper()


In [ ]:
# Fetching the data.
def fetch_energy_data(indicators, years=YEARS, countries=COUNTRIES):
    url = FETCH_ENDPOINT
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json, text/plain, */*",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": BASE_URL,
        "Referer": f"{BASE_URL}/database",
    }

    payload = {
        "mainGroup": "Electricity",
        "mainIndicator[]": ["Access", "Supply", "Technical"],
        "mainIndicatorValue[]": indicators,
        "year[]": years,
        "name[]": countries,
    }

    print(f"Fetching {len(indicators)} indicators for {len(countries)} countries...")
    response = scraper.post(url, headers=headers, data=payload)
    print("Status:", response.status_code)

    try:
        return response.json()
    except Exception as e:
        print("Error decoding JSON:", e)
        print(response.text[:500])
        return []


In [4]:
indicators = [
    "Population access to electricity-National (% of population)",
    "Population access to electricity-Rural (% of population)",
    "Population access to electricity-Urban (% of population)",
    "Electricity generation, Total (GWh)",
    "Electricity generated from hydropower (GWh)",
    "Electricity installed capacity, Total (MW)"
]

raw_data = fetch_energy_data(indicators)
print("Records fetched:", len(raw_data))


Fetching 6 indicators for 54 countries...
Status: 200
Records fetched: 6


In [ ]:
# Transforming the data.
def transform_data(raw_data):
    rows = []
    for block in raw_data:
        metric = block.get("_id")
        for record in block.get("data", []):
            rows.append({
                "country": record.get("name"),
                "country_serial": record.get("id"),
                "metric": metric,
                "unit": record.get("unit"),
                "sector": record.get("indicator_group"),
                "sub_sector": record.get("indicator_topic"),
                "sub_sub_sector": record.get("indicator_name"),
                "source_link": BASE_URL + record.get("url", ""),
                "source": record.get("indicator_source"),
                "year": record.get("year"),
                "value": record.get("score"),
            })

    df = pd.DataFrame(rows)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    df_wide = df.pivot_table(
        index=["country","country_serial","metric","unit","sector",
               "sub_sector","sub_sub_sector","source_link","source"],
        columns="year", values="value", aggfunc="first"
    ).reset_index()

    # Add missing year columns (2000–2024)
    for y in YEARS:
        if y not in df_wide.columns:
            df_wide[y] = None

    df_wide.columns = [str(c) for c in df_wide.columns]
    return df_wide


In [8]:
df_wide = transform_data(raw_data)
df_wide.head(10)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,DZ,Electricity generated from hydropower (GWh),GWh,Electricity,Supply,Electricity generated from hydropower (GWh),https://africa-energy-portal.org/aep/country/a...,AFREC Database,54.000000,...,145.000000,218.000000,56.299999,117.000000,119.674599,122.345169,9.300000,NaN,None,None
1,Algeria,DZ,"Electricity generation, Total (GWh)",GWh,Electricity,Supply,"Electricity generation, Total (GWh)",https://africa-energy-portal.org/aep/country/a...,AFREC Database,25412.000000,...,68798.000000,70997.000000,76017.359375,76663.093750,80234.656250,84104.476562,85390.351562,NaN,None,None
2,Algeria,DZ,"Electricity installed capacity, Total (MW)",MW,Electricity,Technical,"Electricity installed capacity, Total (MW)",https://africa-energy-portal.org/aep/country/a...,IRENA database,6122.810059,...,18032.630859,20073.300781,20588.199219,22188.000000,24527.599609,25348.199219,25724.500000,26609.199219,None,None
3,Angola,AO,Electricity generated from hydropower (GWh),GWh,Electricity,Supply,Electricity generated from hydropower (GWh),https://africa-energy-portal.org/aep/country/a...,AFREC Database,912.000000,...,5192.000000,5815.000000,7653.076660,10374.320312,10754.787109,11187.319336,11675.419922,NaN,None,None
4,Angola,AO,"Electricity generation, Total (GWh)",GWh,Electricity,Supply,"Electricity generation, Total (GWh)",https://africa-energy-portal.org/aep/country/a...,AFREC Database,1445.000000,...,9764.000000,10361.000000,10719.839844,13076.566406,13506.596680,13990.863281,14532.904297,NaN,None,None
5,Angola,AO,"Electricity installed capacity, Total (MW)",MW,Electricity,Technical,"Electricity installed capacity, Total (MW)",https://africa-energy-portal.org/aep/country/a...,IRENA database,429.778992,...,2397.014893,3136.282959,4781.559082,5449.571777,5804.500977,6138.508789,6138.576172,6422.595215,None,None
6,Benin,BJ,Electricity generated from hydropower (GWh),GWh,Electricity,Supply,Electricity generated from hydropower (GWh),https://africa-energy-portal.org/aep/country/b...,AFREC Database,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,None,None
7,Benin,BJ,"Electricity generation, Total (GWh)",GWh,Electricity,Supply,"Electricity generation, Total (GWh)",https://africa-energy-portal.org/aep/country/b...,AFREC Database,134.000000,...,329.010681,341.021423,330.658051,223.063828,227.888596,232.831375,933.147949,NaN,None,None
8,Benin,BJ,"Electricity installed capacity, Total (MW)",MW,Electricity,Technical,"Electricity installed capacity, Total (MW)",https://africa-energy-portal.org/aep/country/b...,IRENA database,56.500000,...,101.699997,203.399994,203.399994,177.399994,304.399994,304.440002,304.440002,329.440002,None,None
9,Botswana,BW,Electricity generated from hydropower (GWh),GWh,Electricity,Supply,Electricity generated from hydropower (GWh),https://africa-energy-portal.org/aep/country/b...,AFREC Database,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,None,None


In [ ]:
# Validating the Data
def validate_data(df):
    missing_years_report = []
    year_cols = [str(y) for y in YEARS]

    for _, row in df.iterrows():
        missing = [y for y in year_cols if pd.isna(row[y])]
        if missing:
            missing_years_report.append({
                "country": row["country"],
                "metric": row["metric"],
                "missing_years": missing
            })

    print(f"Total rows with missing years: {len(missing_years_report)}")
    return missing_years_report

gaps = validate_data(df_wide)


Total rows with missing years: 162


In [ ]:
#Eporting to csv
import os
from datetime import datetime

save_path = r"C:\Users\David\Luxdev\energy scraper\energy scraper 2"


os.makedirs(save_path, exist_ok=True)


timestamp = datetime.now().strftime("%Y%m%d_%H%M")
file_name = f"africa_energy_data_{timestamp}.csv"


full_csv_path = os.path.join(save_path, file_name)


df_wide.to_csv(full_csv_path, index=False, encoding="utf-8-sig")



In [ ]:
# Connecting with MongoDB
from pymongo import MongoClient
from dotenv import load_dotenv
import os


load_dotenv()


mongo_uri = os.getenv("MONGO_URI")


client = MongoClient(mongo_uri)


db = client["energydb"]          
collection = db["energydata"]    

print("Connected to MongoDB successfully!")


Connected to MongoDB successfully!


In [13]:
print(client.list_database_names())


['admin', 'config', 'local']


In [14]:
data = df_wide.to_dict("records")
collection.insert_many(data)
print(f"Inserted {len(data)} records into MongoDB.")


Inserted 162 records into MongoDB.
